## Basic setting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

In [ ]:
%%time
#Everyone needs to save the dataset in their own drive.
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/
CPU times: user 653 ms, sys: 118 ms, total: 772 ms
Wall time: 15 s


In [ ]:
%%time
complaints = pd.read_csv('/content/gdrive/MyDrive/complaints.csv')

<timed exec>:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 29 s, sys: 3.28 s, total: 32.3 s
Wall time: 45.1 s


In [ ]:
# Drop rows where 'Consumer complaint narrative' is null
complaints = complaints.dropna(subset=['Consumer complaint narrative'])

In [ ]:
complaints.isnull().sum()

Date received                         0
Product                               0
Sub-product                       52207
Issue                                 0
Sub-issue                        214653
Consumer complaint narrative          0
Company public response          717059
Company                               0
State                              5542
ZIP code                              0
Tags                            1275553
Consumer consent provided?            0
Submitted via                         0
Date sent to company                  0
Company response to consumer          2
Timely response?                      0
Consumer disputed?              1315455
Complaint ID                          0
dtype: int64

In [ ]:
complaints_revised = complaints[['Company', 'Consumer complaint narrative']]

In [ ]:
#Choose five banks for sample
company_list = ['WELLS FARGO & COMPANY', 'BANK OF AMERICA, NATIONAL ASSOCIATION', 'CAPITAL ONE FINANCIAL CORPORATION', 'JPMORGAN CHASE $ CO.', 'CITIBANK, N.A.']
complaints_revised = complaints_revised[complaints_revised['Company'].isin(company_list)]
complaints_revised.head(10)

,Company,Consumer complaint narrative
5,"CITIBANK, N.A.",Citibank allowed debit card transactions to ov...
37,"BANK OF AMERICA, NATIONAL ASSOCIATION","On XX/XX/, 2023, XXXX XXXX admitted liability ..."
61,WELLS FARGO & COMPANY,My address on all my and my wife 's accounts a...
102,"CITIBANK, N.A.",I am writing to file a complaint against Citib...
112,"CITIBANK, N.A.",Good Afternoon I am submitting a complaint aga...
145,CAPITAL ONE FINANCIAL CORPORATION,"XX/XX/ XXXX XXXX XXXX XXXX XXXX XXXX, Fl XXXX ..."
704,WELLS FARGO & COMPANY,The existence of a derogatory rating on my acc...
836,"CITIBANK, N.A.",I am writing to file a formal complaint concer...
1006,"CITIBANK, N.A.",I had a Banking relationship for XXXX years w...
1012,"CITIBANK, N.A.",I asked Citibank to close my accounts and retu...


In [ ]:
complaints_revised.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142328 entries, 5 to 4080943
Data columns (total 2 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Company                       142328 non-null  object
 1   Consumer complaint narrative  142328 non-null  object
dtypes: object(2)
memory usage: 3.3+ MB


In [ ]:
import random

# Specify the sample size you want - 50
sample_size = 50

# Extract the 'Consumer complaint narrative' column as a list
narratives = complaints_revised['Consumer complaint narrative'].tolist()

# Check if the number of narratives is greater than the desired sample size
if len(narratives) > sample_size:
    random_samples = random.sample(narratives, sample_size)
else:
    random_samples = narratives

# Create a DataFrame from the sampled narratives
sample = pd.DataFrame({'Consumer complaint narrative': random_samples})

In [ ]:
sample.head(10)

,Consumer complaint narrative
0,Capital one is attempting to collect on an exp...
1,On XXXX XXXX I attempted to send {$1000.00} to...
2,Costco decided to change credit card company s...
3,I posted my credit card payment the day it was...
4,Received letter from Wells Fargo thanking me f...
5,group stalking used for illegal collection in ...
6,I have a current mortgage with Wells Fargo. Th...
7,I have a loan with CAPITALONE. I have always m...
8,CAPITAL ONE CC # XXXX HAS BEEN PAID OFF > STIL...
9,I received an email from someone named XXXX ( ...


In [ ]:
sample.isnull().sum()

Consumer complaint narrative    0
dtype: int64

Red Pajama set up

## ## LLaMA 2 on Together API

In [ ]:
!nvidia-smi

Tue Oct 31 19:22:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install --upgrade together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
import os

os.environ["TOGETHER_API_KEY"] = "80d94b40415d1d9de315a72345a3b1cdf5ff2bd3417cd4d119fa56159ad2ef28"


In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.327
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Setting up Together API

In [ ]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

# print the first model's name
print(models[3]['name']), print(models[52]['name'])

EleutherAI/pythia-1b-v0
togethercomputer/CodeLlama-7b


(None, None)

In [ ]:
for idx, model in enumerate(models):
    print(idx, model['name'])

0 Austism/chronos-hermes-13b
1 EleutherAI/llemma_7b
2 EleutherAI/pythia-12b-v0
3 EleutherAI/pythia-1b-v0
4 EleutherAI/pythia-2.8b-v0
5 EleutherAI/pythia-6.9b
6 Gryphe/MythoMax-L2-13b
7 HuggingFaceH4/starchat-alpha
8 NousResearch/Nous-Hermes-13b
9 NousResearch/Nous-Hermes-Llama2-13b
10 NousResearch/Nous-Hermes-Llama2-70b
11 NousResearch/Nous-Hermes-llama-2-7b
12 NumbersStation/nsql-llama-2-7B
13 Open-Orca/Mistral-7B-OpenOrca
14 OpenAssistant/llama2-70b-oasst-sft-v10
15 OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5
16 OpenAssistant/stablelm-7b-sft-v7-epoch-3
17 Phind/Phind-CodeLlama-34B-Python-v1
18 Phind/Phind-CodeLlama-34B-v2
19 SG161222/Realistic_Vision_V3.0_VAE
20 WizardLM/WizardCoder-15B-V1.0
21 WizardLM/WizardCoder-Python-34B-V1.0
22 WizardLM/WizardLM-70B-V1.0
23 bigcode/starcoder
24 databricks/dolly-v2-12b
25 databricks/dolly-v2-3b
26 databricks/dolly-v2-7b
27 defog/sqlcoder
28 garage-bAInd/Platypus2-70B-instruct
29 huggyllama/llama-13b
30 huggyllama/llama-30b
31 huggyllama/llama

In [ ]:
together.Models.start("togethercomputer/RedPajama-INCITE-7B-Chat")

{'success': True,
 'value': '8f3cafdc460080e0cba1188579f0cf99fd0f9370414e94a35f003c3ce5f21db8-5fef59bc92b5044aa0ddf87896f698aacd7ef75ea74bdf08a315f8e3fd57b2ae'}

In [ ]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/RedPajama-INCITE-7B-Chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [ ]:
from langchain import PromptTemplate,  LLMChain

In [ ]:
llm = TogetherLLM(
    model= "togethercomputer/RedPajama-INCITE-7B-Chat",
    temperature=0.1,
    max_tokens=512
)

In [ ]:
llm('"What is Llama 2 from Meta? ')

'\n"Llama 2 is a large language model developed by Meta Platforms. It is designed to be able to process and generate text at a level similar to a human, making it ideal for use in various artificial intelligence applications such as chatbots, speech recognition, and natural language processing.\n\n"What makes Llama 2 special? \n"Unlike other large language models, Llama 2 is designed to be fast and efficient, making it suitable for use in real-time applications. It is also designed to be more transparent than other models, allowing users to better understand how it works and how to use it effectively.\n\n"What are the limitations of Llama 2? \n"Like other large language models, Llama 2 has some limitations. One of the main limitations is that it is trained on a large amount of text from the internet, which means that it has learned to generate text that is similar to what is found on the internet. This can lead to some bias in the generated text.\n\n"How can I use Llama 2 in my own pro

In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text

## Synthesize Complaints



* The initial exploratory analysis identified three types of complaints:
 * Complaints that are generally well written - DESIRABLE
 * Complaints with very poor grammar and punctuation - PROBLEMATIC
 * Complaints with frivolous, unrelated information - PROBLEMATIC



Based on this, we had this idea: Have we focus on just the invariance and robustness aspects related to these “problematic” complaints and focus on questions like “Can the right LLM with the right prompt ‘clean up’ these problematic complaints?” In other words, can an LLM correct the poor grammar and punctuation without changing the meaning (invariance) and can an LLM identify the frivolous content of a complaint (robustness)?





Sample Complaints - Lets see how it looks like

In [ ]:
random_samples[:5]

['Capital one is attempting to collect on an expired debt via a judgement and threatening to garnish my wages. My rights as a federally protected consumer were violated. On XX/XX/21 I sent Capital One a cease and desist demanding all communication stop immediately unless it was in regards to my remedy. Pursuant to 15 usc 1692 ( c ) ( a ) Without the prior consent of the consumer given directly to the debt collector or the express permission of a court of com- petent jurisdiction, a debt collector may not communicate with a consumer in connection with the collection of any debt. I never gave consent and no consent is fraud. Also pursuant to 15 usc 1692 ( d ) ( 1 ) A debt collector may not engage in any conduct the natu- ral consequence of which is to harass, oppress, or abuse any person in connection with the collection of a debt. Without limiting the general application of the foregoing, the following conduct is a violation of this section : ( 1 ) The use or threat of use of violence o

Prompt for Robustness Testing

In [ ]:
system_prompt = "You are a quality control specialist tasked with identifying grammar errors in customer feedback."
instruction = "Evaluate the clarity and coherence of the writing in the following customer complaints :\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>
You are a quality control specialist tasked with identifying grammar errors in customer feedback.
<</SYS>>

Evaluate the clarity and coherence of the writing in the following customer complaints :

 {text}[/INST]


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
for i, complaint in enumerate(random_samples[0:5]):
    examination_result = llm_chain.run(complaint)
    print(i, examination_result)

0 
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGrammarError>
<</QGra

As we can see some complaints have issues related to grammar. Let's see if LLM can change complaints look better.

Prompt for Equal Complaint Synthesis (MFT)

In [ ]:
system_prompt = "You are a natural language generator tasked with producing customer feedback."
instruction = "Generate a customer complaint that is grammatically correct and equivalent in meaning to the given text, using a different wording and in 50 words or less. :\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>
You are a natural language generator tasked with producing customer feedback.
<</SYS>>

Generate a customer complaint that is grammatically correct and equivalent in meaning to the given text, using a different wording and in 50 words or less. :

 {text}[/INST]


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
for i, complaint in enumerate(random_samples[0:5]):
    examination_result = llm_chain.run(complaint)
    print(i, examination_result)

0  Sure, here's a customer complaint that is grammatically correct and equivalent in meaning to the given text, using a different wording and in 50 words or less:

"I recently discovered two unauthorized inquiries on my credit report from CBNA, despite never contacting them. This is a frustrating mistake, especially since I've worked hard to improve my credit. I don't know where to start to resolve this issue, but I will do whatever it takes to have it removed."
1  "Fraudsters posing as Capital One tricked me into transferring over $2500.00. I just discovered my personal info is on the dark web after calling XXXX to open a new account. They told me someone used my SSN to create a fake account. I've canceled it and asked XXXX to flag it to prevent further fraud. The transfers were made to XXXX XXXX on my XXXX file."
2  "On 20/02/2020, I was surprised to learn from Bank of America that my account was being used for unauthorized purchases on an online gaming site called XXXX."
3  Sure, he

Prompt for Invariance Complaint Synthesis

In [ ]:
system_prompt = "You are a text-to-speech system that converts written complaints into spoken language."
instruction = "Convert the given text into a spoken complaint that sounds natural and authentic, while conveying the same message and in 50 words or less. :\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>
You are a text-to-speech system that converts written complaints into spoken language.
<</SYS>>

Convert the given text into a spoken complaint that sounds natural and authentic, while conveying the same message and in 50 words or less. :

 {text}[/INST]


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
for i, complaint in enumerate(random_samples[0:5]):
    examination_result = llm_chain.run(complaint)
    print(i, examination_result)

0  Here is a spoken complaint that conveys the same message in 50 words or less:

"I have two recent inquiries on my credit report from CBNA, despite never contacting them. This is fraudulent and I demand it be removed. I've worked hard on my credit and don't need false information. I'm not sure where to start or what to say to CBNA, but I will take action to resolve this issue."
1  "Someone used my personal information to open a bank account in my name and transferred over $2500. I just called the bank to report the fraud and they told me my personal information is on the dark web. I've changed my phone number and am deleting my old email account. The fraudulent transfers were made to a different bank account." (48 words)
2  "On February 20th, I was shocked to learn from Bank of America that my account was being used to make unauthorized purchases on an online gaming site, XXXX. This is unacceptable and I demand immediate action to protect my account and personal information."
3  "I'm

Prompt for Harshness Modification (Direction):

In [ ]:
system_prompt = "You are a natural language generator tasked with producing customer feedback."
instruction = "Generate a new complaint that sounds less harsh and confrontational than the original complaint. Use a polite and conciliatory tone to make the complaint sound more collaborative and less aggressive (50 words). :\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>
You are a natural language generator tasked with producing customer feedback.
<</SYS>>

Generate a new complaint that sounds less harsh and confrontational than the original complaint. Use a polite and conciliatory tone to make the complaint sound more collaborative and less aggressive (50 words). :

 {text}[/INST]


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
for i, complaint in enumerate(random_samples[0:5]):
    examination_result = llm_chain.run(complaint)
    print(i, examination_result)

0 
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<</LANG>>
<</CS>>
<
1 
<</LANGUAGE>
<</POLITE>
<</CONCILIATORY>
<</POLITE>
<</CONCILIATORY>
<</PO

In [ ]:
system_prompt = "You are a natural language generator tasked with producing customer feedback."
instruction =  "Generate a new complaint that sounds more harsh and confrontational than the original complaint. Use an aggressive and forceful tone to make the complaint sound more impactful and urgent (50 words). :\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>
You are a natural language generator tasked with producing customer feedback.
<</SYS>>

Generate a new complaint that sounds more harsh and confrontational than the original complaint. Use an aggressive and forceful tone to make the complaint sound more impactful and urgent (50 words). :

 {text}[/INST]


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
for i, complaint in enumerate(random_samples[0:5]):
    examination_result = llm_chain.run(complaint)
    print(i, examination_result)

0  Here is a new complaint that sounds more harsh and confrontational than the original:

"I am outraged and appalled by the recent inquiries made on my credit report by CBNA, despite my explicit denial of any authorization. Not only is this a blatant violation of my privacy and security, but it also undermines my hard-earned credit score. I demand immediate action to remove these fraudulent inquiries and protect my personal information. I will not hesitate to take legal action if necessary, and I expect a prompt resolution to this matter. How dare you attempt to sully my good name and creditworthiness? I will not be ignored!"
1  "Are you kidding me?! I can't believe I was duped by some impersonator claiming to be from Capital One and they got me to transfer over $2500! And now, to make matters worse, I find out that someone used my SSN to create a fake account in my name?! I demand that you take immediate action to flag my SSN and prevent any further fraudulent activity. And don't eve

## Disconnect server  

In [ ]:
together.Models.stop("togethercomputer/falcon-7b")

{'success': True}